In [2]:
%env SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
%env HF_HUB_DISABLE_SYMLINKS_WARNING=True
!pip install -q transformers opendelta peft

env: SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
env: HF_HUB_DISABLE_SYMLINKS_WARNING=True
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.26.100 requires botocore<1.30.0,>=1.29.100, but you have botocore 1.34.34 which is incompatible.


In [3]:
import logging
import wandb
import torch

wandb.login(key='')
logging.disable(logging.WARNING)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import pandas as pd
from transformers import Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from opendelta import PrefixModel, Visualization
from sklearn.model_selection import train_test_split

train_path = '/kaggle/input/boolq-slobench-mt/train.csv'
test_path = '/kaggle/input/boolq-slobench-mt/test.csv'
val_path = '/kaggle/input/boolq-slobench-mt/val.csv'

# Load each CSV file into a DataFrame
train_df = pd.read_csv(train_path).drop('idx', axis=1)
test_df = pd.read_csv(test_path).drop('idx', axis=1)
val_df = pd.read_csv(val_path).drop('idx', axis=1)

train_df, labelled_test_df = train_test_split(
    train_df, 
    test_size=0.2,  
    random_state=42
)

print(labelled_test_df.head())

2024-03-22 08:30:10.891652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 08:30:10.891798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 08:30:11.014677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


      label                                            passage  \
8681   True  Jurassic World: Fallen Kingdom - Snemanje je p...   
2362  False  Prekletstvo Super Bowla - prekletstvo domačega...   
6232   True  Radarski detektor - radarski motilci po zvezni...   
1318   True  Chitty Chitty Bang Bang - Chitty Chitty Bang B...   
543    True  Grand Slam (golf) - Izraz &quot;Grand Slam&quo...   

                                               question  
8681      je juraško svetovno padlo kraljestvo še izšlo  
2362     je kdaj v super skledi bila gostiteljska ekipa  
6232  ali je zakonito imeti radarski detektor v Teks...  
1318             je chitty chitty bang bang samo zgodba  
543       je kdo v enem letu osvojil grand slam v golfu  


In [5]:
# Converting DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
additional_test_dataset = Dataset.from_pandas(labelled_test_df)
existing_test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [6]:
model_name = "EMBEDDIA/sloberta"  
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) 
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

In [7]:
from peft import PrefixTuningConfig, get_peft_model

peft_config = PrefixTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=20)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
print(model)

trainable params: 960,770 || all params: 111,584,260 || trainable%: 0.861026456598807
PeftModelForSequenceClassification(
  (base_model): CamembertForSequenceClassification(
    (roberta): CamembertModel(
      (embeddings): CamembertEmbeddings(
        (word_embeddings): Embedding(32005, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): CamembertEncoder(
        (layer): ModuleList(
          (0-11): 12 x CamembertLayer(
            (attention): CamembertAttention(
              (self): CamembertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
               

In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

def tokenize_function(examples):
    return tokenizer(examples['passage'], examples['question'], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = additional_test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
print(test_dataset)

Dataset({
    features: ['label', 'passage', 'question', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1886
})


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: camile-lendering (ul-nlp). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.659800,0.663552,0.621713,0.766736,0.621713,1.000000
2,0.654800,0.669188,0.621713,0.766736,0.621713,1.000000
3,0.656700,0.661992,0.621713,0.766736,0.621713,1.000000


TrainOutput(global_step=11313, training_loss=0.6640970113264159, metrics={'train_runtime': 721.4589, 'train_samples_per_second': 31.357, 'train_steps_per_second': 15.681, 'total_flos': 2996756585441280.0, 'train_loss': 0.6640970113264159, 'epoch': 3.0})

In [12]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 0.6590494513511658, 'eval_accuracy': 0.6251325556733828, 'eval_f1': 0.7693311582381729, 'eval_precision': 0.6251325556733828, 'eval_recall': 1.0, 'eval_runtime': 21.0718, 'eval_samples_per_second': 89.503, 'eval_steps_per_second': 44.752, 'epoch': 3.0}


In [14]:
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/sentencepiece.bpe.model',
 './results/added_tokens.json',
 './results/tokenizer.json')